## Imports and logging

In [2]:
import os
import re
import json
import jsonschema
import pandas as pd
import numpy as np
import logging
logging.basicConfig()
log = logging.getLogger('U2787g')

## Functions

In [51]:
def validate_against_template(input_dict, template, error_log_path):
    """
    This is a function for validating a dictionary against a template. Given
    an input_dict and a template object, it will create a JSON schema validator
    and construct an object that is a list of error dictionaries. It will write a
    JSON file to the specified error_log_path and return the validation_errors object as
    well as log each error.message to log.errors

    :param input_dict: a dictionary of DICOM header data to be validated
    :param template: a template dictionary to validate against
    :param error_log_path: the path to which to write error log JSON
    :return: validation_errors, an object containing information on validation errors
    """
    # Initialize json schema validator
    validator = jsonschema.Draft7Validator(template)
    # Initialize list object for storing validation errors
    validation_errors = []
    for error in sorted(validator.iter_errors(input_dict), key=str):
        # Create a temporary dictionary for the individual error
        tmp_dict = {}
        # Get error type
        tmp_dict['error_type'] = error.validator
        # Get error message and log it
        tmp_dict['error_message'] = error.message
        log.error(error.message)
        # Required field errors are a little special and need to be handled
        # separately to get the field. We don't get the schema because it
        # will print the entire template schema
        if error.validator == "required":
            # Get the item failing validation from the error message
            tmp_dict['item'] = 'info.' + error.message.split("'")[1]
        # Get additional information for pattern and type errors
        elif error.validator in ("pattern", "type"):
            # Get the value of the field that failed validation
            tmp_dict['error_value'] = error.instance
            # Get the field that failed validation
            tmp_dict['item'] = 'info.' + str(error.path.pop())
            # Get the schema object used to validate in failed validation
            tmp_dict['schema'] = error.schema
        elif error.validator == "anyOf":
            tmp_dict['schema'] = {"anyOf": {error.schema['anyOf']}}
        else:
            pass
        # Append individual error object to the return validation_errors object
        validation_errors.append(tmp_dict)

    with open(error_log_path, 'w') as outfile:
        json.dump(validation_errors, outfile, separators=(', ', ': '), sort_keys=True, indent=4)
    return validation_errors

## Load in JSON Schema template

In [16]:
template_filepath = "U2787g_template"

with open(template_filepath) as template_data:
    template_json = json.load(template_data)

In [22]:
test_dictionary = {
    "Modality": "CT",
    "ImageType": "SCREEN SAVE",
    "PatientID": "55555",
}

In [52]:
validate_against_template(test_dictionary, template_json, "error.log.json")

ERROR:U2787g:'CT' does not match 'MR'
ERROR:U2787g:'SCREEN SAVE' does not match '^(?!SCREEN SAVE).*$'
ERROR:U2787g:{'Modality': 'CT', 'ImageType': 'SCREEN SAVE', 'PatientID': '55555'} is not valid under any of the given schemas


TypeError: unhashable type: 'list'

In [20]:
# Initialize json schema validator
validator = jsonschema.Draft7Validator(template_json)

In [48]:
for error in sorted(validator.iter_errors(test_dictionary), key=str):
    print(error.validator)
    if error.validator == "anyOf":
        print(error.schema)

pattern
pattern
anyOf
{'properties': {'ImageType': {'description': "ImageType cannot be 'SCREEN SAVE'", 'type': 'string', 'pattern': '^(?!SCREEN SAVE).*$'}, 'Modality': {'description': "Modality must match 'MR'", 'pattern': 'MR', 'type': 'string'}, 'PatientID': {'description': 'PatientID must be 5 numeric characters', 'pattern': '^[0-9]{5}$', 'type': 'string'}}, 'type': 'object', 'anyOf': [{'required': ['AcquisitionDate']}, {'required': ['SeriesDate']}, {'required': ['StudyDate']}]}
